In [96]:
import pandas as pd
# Load the file to inspect its structure
df = pd.read_csv('2-3Apr X scrape.csv', encoding='ISO-8859-1')
df.head()

,ï»¿createdAt,fullText,author_id,author_location
0,Thu Apr 03 23:59:57 +0000 2025,@SpencerHakimian Tariffs arenât self-taxatio...,1487236076548894724,"Issaquah, WA"
1,Thu Apr 03 23:59:56 +0000 2025,@joma_gc Veitnam has the audacity to have a 90...,1762610631348510720,NaN
2,Thu Apr 03 23:59:47 +0000 2025,My thoughts on the tariff debate:\n\n1) If the...,1126395138,California
3,Thu Apr 03 23:59:44 +0000 2025,@thejefflutz Grok has the total labor hours to...,4764020911,"Kenmore, WA"
4,Thu Apr 03 23:59:44 +0000 2025,"'India very, very tough. The Prime Minister ju...",1790517608153772033,Los Angeles California


In [98]:
import re

def clean_text_preprocessing1(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)     # Remove URLs
    text = re.sub(r"@\w+|#\w+", "", text)                   # Remove mentions and hashtags
    text = re.sub(r"[^\x00-\x7F]+", "", text)               # Remove emojis (non-ASCII characters)
    text = re.sub(r"\d+", "", text)                         # Remove numerical characters
    return text

# Apply this to DataFrame 
df['text_preprocessing1'] = df['fullText'].apply(clean_text_preprocessing1)
df[['fullText', 'text_preprocessing1']].head()

,fullText,text_preprocessing1
0,@SpencerHakimian Tariffs arenât self-taxatio...,"Tariffs arent self-taxation, theyre leverage...."
1,@joma_gc Veitnam has the audacity to have a 90...,Veitnam has the audacity to have a % tariff o...
2,My thoughts on the tariff debate:\n\n1) If the...,My thoughts on the tariff debate:\n\n) If they...
3,@thejefflutz Grok has the total labor hours to...,Grok has the total labor hours to build a car...
4,"'India very, very tough. The Prime Minister ju...","'India very, very tough. The Prime Minister ju..."


In [100]:
import string

def clean_text_preprocessing2(text):
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation
    return text

# Apply this to the result of step 1
df['text_preprocessing_no_punctuation'] = df['text_preprocessing1'].apply(clean_text_preprocessing2)
df[['fullText', 'text_preprocessing1','text_preprocessing_no_punctuation']].head()

,fullText,text_preprocessing1,text_preprocessing_no_punctuation
0,@SpencerHakimian Tariffs arenât self-taxatio...,"Tariffs arent self-taxation, theyre leverage....",Tariffs arent selftaxation theyre leverage If...
1,@joma_gc Veitnam has the audacity to have a 90...,Veitnam has the audacity to have a % tariff o...,Veitnam has the audacity to have a tariff on...
2,My thoughts on the tariff debate:\n\n1) If the...,My thoughts on the tariff debate:\n\n) If they...,My thoughts on the tariff debate\n\n If they t...
3,@thejefflutz Grok has the total labor hours to...,Grok has the total labor hours to build a car...,Grok has the total labor hours to build a car...
4,"'India very, very tough. The Prime Minister ju...","'India very, very tough. The Prime Minister ju...",India very very tough The Prime Minister just ...


In [102]:
# Convert texts to lowercase
df['text_preprocessing_no_punctuation'] = \
df['text_preprocessing_no_punctuation'].map(lambda x: x.lower())

df['text_preprocessing_no_punctuation'].head()

0     tariffs arent selftaxation theyre leverage if...
1     veitnam has the audacity to have a  tariff on...
2    my thoughts on the tariff debate\n\n if they t...
3     grok has the total labor hours to build a car...
4    india very very tough the prime minister just ...
Name: text_preprocessing_no_punctuation, dtype: object

In [104]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in ENGLISH_STOP_WORDS]
    return " ".join(filtered)

# Apply to cleaned column
df['text_no_stopwords'] = df['text_preprocessing_no_punctuation'].apply(remove_stopwords)
df[['text_preprocessing_no_punctuation', 'text_no_stopwords']].head()

,text_preprocessing_no_punctuation,text_no_stopwords
0,tariffs arent selftaxation theyre leverage if...,tariffs arent selftaxation theyre leverage vie...
1,veitnam has the audacity to have a tariff on...,veitnam audacity tariff goods f
2,my thoughts on the tariff debate\n\n if they t...,thoughts tariff debate tariff law equality dic...
3,grok has the total labor hours to build a car...,grok total labor hours build car parts hr that...
4,india very very tough the prime minister just ...,india tough prime minister just left great fri...


In [106]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

# Download required resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
        
def lemmatize_with_pos(text):
    tokens = nltk.word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lemmatized = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos)) 
        for word, pos in tagged_tokens
    ]
    return " ".join(lemmatized)
    
df['text_lemmatized'] = df['text_no_stopwords'].apply(lemmatize_with_pos)
df[['text_no_stopwords', 'text_lemmatized']].head()

[nltk_data] Downloading package punkt to /Users/patnbe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/patnbe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/patnbe/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/patnbe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text_no_stopwords,text_lemmatized
0,tariffs arent selftaxation theyre leverage vie...,tariff arent selftaxation theyre leverage viet...
1,veitnam audacity tariff goods f,veitnam audacity tariff good f
2,thoughts tariff debate tariff law equality dic...,thought tariff debate tariff law equality dict...
3,grok total labor hours build car parts hr that...,grok total labor hour build car part hr thats ...
4,india tough prime minister just left great fri...,india tough prime minister just leave great fr...


In [28]:
# Export cleaned_data.csv file for using in BERTopic model that is performed in Google Colab due to the conflict in versions of some libraries used in both models
df.to_csv('cleaned_data.csv', index=False)

In [44]:
# Tokenize text
tokenized_text = df['text_lemmatized'].apply(lambda x: x.split())

In [46]:
from gensim import corpora

# Create Dictionary
id2word = corpora.Dictionary(tokenized_text)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in tokenized_text]

In [86]:
from gensim.models.ldamodel import LdaModel

# Train LDA model
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=2, # number of topics
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

In [88]:
# Display topics (word distribution in each topic)
topics = lda_model.print_topics()
for idx, topic in topics:
    print(f"Topic {idx + 1}: {topic}")

Topic 1: 0.072*"tariff" + 0.038*"trump" + 0.015*"trade" + 0.013*"import" + 0.011*"market" + 0.010*"good" + 0.009*"india" + 0.009*"reciprocal" + 0.008*"president" + 0.007*"donald"
Topic 2: 0.032*"tariff" + 0.010*"amp" + 0.008*"country" + 0.008*"trump" + 0.008*"pay" + 0.006*"dont" + 0.005*"america" + 0.005*"american" + 0.005*"tax" + 0.005*"list"


In [90]:
# Display topic distribution in each document
lda_model.get_document_topics(corpus)
for i, doc in enumerate(corpus):
    topics = lda_model.get_document_topics(doc)
    print(f"Document {i + 1} Topics: {topics}")

Document 1 Topics: [(0, 0.9953134)]
Document 2 Topics: [(0, 0.32878932), (1, 0.6712107)]
Document 3 Topics: [(0, 0.3279594), (1, 0.6720406)]
Document 4 Topics: [(0, 0.014146137), (1, 0.9858539)]
Document 5 Topics: [(0, 0.99338263)]
Document 6 Topics: [(0, 0.99426305)]
Document 7 Topics: [(0, 0.039350882), (1, 0.96064913)]
Document 8 Topics: [(0, 0.98769003), (1, 0.012310003)]
Document 9 Topics: [(0, 0.9940093)]
Document 10 Topics: [(0, 0.15886958), (1, 0.84113044)]
Document 11 Topics: [(0, 0.99400556)]
Document 12 Topics: [(0, 0.029895151), (1, 0.9701048)]
Document 13 Topics: [(0, 0.36418653), (1, 0.6358135)]
Document 14 Topics: [(0, 0.37639248), (1, 0.62360746)]
Document 15 Topics: [(0, 0.9847888), (1, 0.015211252)]
Document 16 Topics: [(0, 0.9904345)]
Document 17 Topics: [(0, 0.9863935), (1, 0.013606503)]
Document 18 Topics: [(0, 0.91729736), (1, 0.082702674)]
Document 19 Topics: [(0, 0.7087635), (1, 0.29123655)]
Document 20 Topics: [(0, 0.99339145)]
Document 21 Topics: [(0, 0.993385

In [92]:
from gensim.models.coherencemodel import CoherenceModel

# Compute c_v coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_text, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print(f"C_v coherence score: {coherence_lda:.4f}")

C_v coherence score: 0.2775


In [94]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualisation
lda_vis = gensimvis.prepare(lda_model, corpus, id2word)

pyLDAvis.display(lda_vis)

/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=3483) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=3483) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=3483) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=3483) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: T